In [ ]:
#this code is taken mostly from https://sefiks.com/2018/01/10/real-time-facial-expression-recognition-on-streaming-data/

In [ ]:
!pip install tensorflow keras

# !!! If you have Windows, this won't work, you have to install Tensorflow and Keras through Anaconda manager

In [ ]:
import numpy as np
import cv2
from keras.preprocessing import image
from keras.models import model_from_json

## Загружаем модель детектора лиц (это еще нейросеть)

In [ ]:
face_cascade = cv2.CascadeClassifier('./haarcascade_frontalface_default.xml')

## Загружаем модель распознавания эмоции (это уже нейросеть)

In [ ]:
model = model_from_json(open("./facial_expression_model_structure.json", "r").read())
model.load_weights('./facial_expression_model_weights.h5') #load weights, warnings are OK

In [ ]:
cap = cv2.VideoCapture(0)
ret, img = cap.read()
cap.release()

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # th model takes 1-channel images
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

In [ ]:
faces

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) #draw rectangle to main image

    detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
    detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
    detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48

    img_pixels = image.img_to_array(detected_face)
    img_pixels = np.expand_dims(img_pixels, axis = 0)

    img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

    predictions = model.predict(img_pixels)[0] #store probabilities of 7 expressions

    #find max indexed array 0: angry, 1:disgust, 2:fear, 3:happy, 4:sad, 5:surprise, 6:neutral
    max_index = np.argmax(predictions)

    emotion = emotions[max_index]

    #write emotion text above rectangle
    cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    #process on detected face end
    #-------------------------

plt.imshow(img[..., ::-1])

посмотрим на цифры

In [ ]:
for i, emotion in enumerate(emotions):
    print("{}: {} ".format(emotion, predictions[i]))

Realtime demo

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) #draw rectangle to main image

        detected_face = img[int(y):int(y+h), int(x):int(x+w)] #crop detected face
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transform to gray scale
        detected_face = cv2.resize(detected_face, (48, 48)) #resize to 48x48

        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)

        img_pixels /= 255 #pixels are in scale of [0, 255]. normalize all pixels in scale of [0, 1]

        predictions = model.predict(img_pixels) #store probabilities of 7 expressions

        #find max indexed array 0: angry, 1:disgust, 2:fear, 3:happy, 4:sad, 5:surprise, 6:neutral
        max_index = np.argmax(predictions[0])

        emotion = emotions[max_index]

        #write emotion text above rectangle
        cv2.putText(img, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)

    cv2.imshow('img',img)

    if cv2.waitKey(1) & 0xFF == ord('q'): #press q to quit
        break

#kill open cv things
cap.release()
cv2.destroyAllWindows()